# **Tutorial 2. Training Models with gRelu and Examining Pitfalls**

"<a href="https://github.com/Genentech/gReLU/blob/main/README.md" target="_blank">gReLU</a>" is a Python library to train, interpret, and apply deep learning models to DNA sequences". As explained in the "Software libraries for model building section", the gRelu library contains a model zoo allowing for easy access to several models such as Borzoi, Enformer, or a dilated convolutional model based on the BPnet model architecture. Borzoi and Enformer can be imported pre-trained. Additionally, simpler base models and convolutional neural networks are also available. <br>

This tutorial begins with how to train models using the gRelu library explaining how the models used in this tutorial were trained. The next stage of this tutorial analyses several pitfalls of machine learning in genomics, using a dilated convolutional network from gRelu trained on the same ChIP-seq experiment from the Encode project used in previous tutorials, <a href="https://www.encodeproject.org/experiments/ENCSR817LUF/" target="_blank">ENCSR817LUF</a>. The models are trained using minimal data (chromosomes 1-5 and chromosome 19) to highlight several pitfalls (distributional differences, leaky-preprocessing, unbalanced classes), and further understand genomic coverage data. 

In the final part of this tutorial, we will look an an example of the effect of differences in sequencing depths in order to understand confounding effects.

In [ ]:
import numpy as np
import pandas as pd
import os
import random
import torch
import grelu.model.models as models
import grelu.data.preprocess
import grelu.data.dataset
import grelu.lightning
import grelu.visualize
import wandb
from pytorch_lightning.loggers import WandbLogger
import pickle
import zipfile
import matplotlib.pyplot as plt
import pyBigWig
import numpy

# **List of gRelu Models Available**

In [ ]:
all_models = dir(models)
model_classes = [item for item in all_models if 'model' in item.lower()]

print("Available Models in gReLU:\n")
for model in model_classes:
    print(model)

# **Training Models with gRelu**

This tutorial focuses on training, understanding, and evaluating gRelu models in order to highlight several pitfalls in the machine learning process. Datasets were made using the code, logic, and scripts from Tutorial 1.5 Preprocessing with gRelu. The code cell below downloads the datasets as well as the models that were trained using those datasets. <br><br>

In [ ]:
!wget -O tutorial_2.zip 'https://www.dropbox.com/scl/fo/jep97l3k09vq6o3zhvmep/APDjOu7UFCGndtKI5kWr5_8?rlkey=j44ilc7nmxatbd00wm7xql8ew&st=lwym0gl0&dl=0'

with zipfile.ZipFile('tutorial_2.zip', 'r') as zip_ref:
    zip_ref.extractall('tutorial_2')

!wget -O datasets.zip 'https://www.dropbox.com/scl/fo/4f7tsjhm4cwg57xbiq5fw/ADqrg8IjAboQWXI6oPo5QY8?rlkey=6u5r0f5yp94js41yi72nthwdh&st=4qugwkb5&dl=0'

with zipfile.ZipFile('datasets.zip', 'r') as zip_ref:
    zip_ref.extractall('datasets')


Training a gRelu Model
-
<br>
As explained in Tutorial 1.5, the datasets were made using a gRelu dataset function, which returns gRelu dataset type object. Training a gRelu model is as simple as defining your gRelu model, model parameters, training parameters and utlising gRelu's 'Lightning.LightningModel' and 'train_on_dataset functions, calling PyTorch's lightning trainer as seen in this section.
<br><br>

In [ ]:
experiment='tutorial_2'
if not os.path.exists(experiment):
    os.makedirs(experiment)

<br>
To explain training a gRelu model, we'll use our balanced datasets. These 'balanced' datasets contain data from chromosomes 1 through 5, with a 50-50 split of peaks and non-peaks. Chromosome 2 and 3 are the test and validation chromosomes respectively, with the rest being used as training data. 
<br><br>
As explained in Tutorial 1.5, the input window is a 2048bp region, while the prediction resolution is 512bps. Coverage values were summed instead of averaged within the 512bps. This means that while the original threshold for a peak was arcsinh(2), what constitutes a peak in our results has changed to arcsinh(1024) or 7.62 as a result of summing over 512bps bins.
<br><br>

In [ ]:

# Load the dataset object from the pickle files
with open('datasets/train_ds_balanced.pkl', 'rb') as f:
    train_ds = pickle.load(f)

with open('datasets/val_ds_balanced.pkl', 'rb') as f:
    val_ds = pickle.load(f)

with open('datasets/test_ds_balanced.pkl', 'rb') as f:
    test_ds_bal = pickle.load(f)


In [ ]:
# Seeding to ensure reproducibility

seed = 42  # or any other integer
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True  
torch.backends.cudnn.benchmark = False     

<br>
A Dilated Convolutional Model will be used, similar to gRelu's <a href="https://genentech.github.io/gReLU/tutorials/3_train.html#" target="_blank">training tutorial</a> on "training a model to predict total coverage over ATAC-seq peaks". For computational and time constraints, the number of channels and number of workers have been reduced. The learning rate has been increased to 1e-3, and an Adam optimiser has been introduced. It is recommended that a GPU be used when training these models.
<br><br>

In [ ]:
INPUT_WINDOW=2048 # Length of the input sequence
PRED_RES=512 # Length over which we calculate total coverage
# val_chroms=["chr3"]
# test_chroms=["chr2"]
# genome="hg38"

model_params = {
    'model_type':'DilatedConvModel',
    'crop_len':(INPUT_WINDOW-PRED_RES)//2, # Central 512bps are used for predictions
    'n_tasks':1,
    'channels':128,
    'n_conv':8,
}

train_params = {
    'task':'regression',
    'loss': 'poisson', # Poisson / mse / poisson_multinomial
    'logger':'csv',
    'lr':1e-3,
    'batch_size':256,
    'max_epochs':10,
    'devices':0,
    'num_workers':0,
    'save_dir':experiment,
    'optimizer': 'adam',
}

import grelu.lightning
model = grelu.lightning.LightningModel(model_params, train_params)

In [ ]:
trainer = model.train_on_dataset(train_ds, val_ds)

In [ ]:
best_checkpoint = trainer.checkpoint_callback.best_model_path
print(best_checkpoint)

In [ ]:
# Loading model from best checkpoint
model = grelu.lightning.LightningModel.load_from_checkpoint(best_checkpoint)

In [ ]:
# Models can be saved using Torch.

#model_path = /path
#torch.save(model, model_path)

Evaluating Models
-
We can utilise gRelu's 'test_on_dataset' and 'predict_on_dataset' functions to test our model on our testset (chromosome 3) and make predictions. We can also visualise the prediction density as a scatter plot using gRelu's visualisation functions.
<br><br>

In [ ]:
# Helper function to look at MSE, RMSE and LogRMSE
def print_stats(preds, labels):
    """
    Calculate and print MSE, RMSE, and Log RMSE.
    
    Parameters:
    - preds: numpy array of predictions.
    - labels: numpy array of actual labels.
    """
    # Mean Squared Error (MSE)
    mse = np.mean((preds - labels.get_labels()) ** 2)
    print(f"MSE: {mse:.3f}")

    # Root Mean Squared Error (RMSE)
    rmse = np.sqrt(mse)
    print(f"RMSE: {rmse:.3f}")

    # Log Root Mean Squared Error (Log RMSE)
    log_rmse = np.log(rmse)
    print(f"LOG RMSE: {log_rmse:.3f}")

In [ ]:
with open('datasets/test_ds_balanced.pkl', 'rb') as f:
    test_ds_bal = pickle.load(f)

test_metrics = model.test_on_dataset(
    test_ds_bal,
    batch_size=256,
    devices=0,
    num_workers=1
)

test_metrics

In [ ]:
preds_bal = model.predict_on_dataset(
    test_ds_bal, devices=0, num_workers=1
    )
preds_bal.shape

In [ ]:
grelu.visualize.plot_pred_scatter(
    preds=((preds_bal)),
    labels=((test_ds_bal.get_labels())),
    density=True, # Color points by local density
    figsize=(3, 2.5), # width, height
    size=.5
)

In [ ]:
print_stats(preds_bal, test_ds_bal)

The All Peaks Model
-
Lets take a look at some of the models that were trained to evaluate the models within the context of the data and pre-processing steps used. Again, the datasets used are available in the 'datasets' folder, made from scripts accessible in Tutorial 1.5. The model, model metrics, tests and predictions are saved in the 'tutorial_2' folder under each model folder for accessibility purposes. This is due to the environment and state the model was saved in, which can cause errors when loading the model in different environments and running gRelu's testing/prediction functions. <br>

Tutorial 1 ended with training, validation and test sets containing thresholded peak regions and their arcsinhed coverage values. How would a model perform if were trained solely on regions of peaks? Firstly lets examine the data distribution below.
<br><br>

In [ ]:
def plot_distribution(labels, title):
    labels_flat = labels.flatten()  # Flatten the labels to 1D
    plt.figure(figsize=(5, 3))
    plt.hist(labels_flat, bins=50, color='skyblue', edgecolor='black')
    plt.title(f'Distribution of {title}')
    plt.xlabel('Target Value')
    plt.ylabel('Frequency')
    plt.show()

with open('datasets/train_ds_allpeaks.pkl', 'rb') as f:
    train_ds_allpeaks = pickle.load(f)
    
with open('datasets/val_ds_allpeaks.pkl', 'rb') as f:
    val_ds_allpeaks = pickle.load(f)
    
with open('datasets/test_ds_allpeaks.pkl', 'rb') as f:
    test_ds_allpeaks = pickle.load(f)
    
train_labels = np.array(train_ds_allpeaks.get_labels())
val_labels = np.array(val_ds_allpeaks.get_labels())
test_labels = np.array(test_ds_allpeaks.get_labels())

In [ ]:
# Plot distributions for each dataset
plot_distribution(train_labels, 'Training Dataset')
plot_distribution(val_labels, 'Validation Dataset')
plot_distribution(test_labels, 'Test Dataset')

<br>
As you can see clearly, the distribution of the each dataset is right skewed. Let's see what issues we may face when training our model on only peak regions. Firstly, lets evaluate the All Peaks model on a test set comprising of only peak data (chromosome 2).
<br><br>

In [ ]:
preds_allpeaks = np.load('tutorial_2/allpeaks_model/version_0/preds_allpeaks.npy')
grelu.visualize.plot_pred_scatter(
    preds=preds_allpeaks,
    labels=test_ds_allpeaks.get_labels(),
    density=True, # Color points by local density
    figsize=(3, 2.5), # width, height
    size=.5
)

In [ ]:
print_stats(preds_allpeaks, test_ds_allpeaks)

<br>
The model performs incredibly well from a mean-squared error standpoint. However, it does fail to recognise higher peak coverage regions, indicating that the model may be underfitting. In trying to predict coverage values, what is more important is to test this model on a real distribution of peaks. How would this model perform? <br>
<br><br>

In [ ]:
with open('datasets/test_ds_real_distribution.pkl', 'rb') as f:
    test_ds_real_distribution = pickle.load(f)
    
preds_apreal_dist = np.load('tutorial_2/allpeaks_model/version_0/preds_real_distribution.npy')
grelu.visualize.plot_pred_scatter(
    preds=(preds_apreal_dist),
    labels=(test_ds_real_distribution.get_labels()),
    density=True, # Color points by local density
    figsize=(3, 2.5), # width, height
    size=.5
)

In [ ]:
print_stats(preds_apreal_dist, test_ds_real_distribution)

<br>
As you could probably predict, when tested on the real distribution of peaks to no peaks in chromosome 2, the model does very poorly with an mse of 13.5. The issue being that the model predicts every region as being a peak, effectively not being useful in trying to differentiate between peak and non-peak regions as it was not trained on any non-peak regions. I highlight this example to complete a continuation of the only peak datasets created in Tutorial 1, and to highlight the issue that peaks are scarce. In the genomic context, many scenarios involve class imbalance where the regions of interest are a minority, in this case peaks. Lets instead try and train our model on datasets containing the real distribution of peaks and non-peaks.
<br><br>

Real Distribution Model
-

In [ ]:
with open('datasets/train_ds_real_distribution.pkl', 'rb') as f:
    train_ds_rd = pickle.load(f)
    
with open('datasets/val_ds_real_distribution.pkl', 'rb') as f:
    val_ds_rd = pickle.load(f)
    
with open('datasets/test_ds_real_distribution.pkl', 'rb') as f:
    test_ds_rd = pickle.load(f)
    
train_labels = np.array(train_ds_rd.get_labels())
val_labels = np.array(val_ds_rd.get_labels())
test_labels = np.array(test_ds_rd.get_labels())

# Plot distributions for each dataset
plot_distribution(train_labels, 'Training Dataset')
plot_distribution(val_labels, 'Validation Dataset')
plot_distribution(test_labels, 'Test Dataset')

<br>
The real distributions of the training set (chr 1,4,5), validation set (chr 3), and test set (chr2) all have a normal distribution and are quite similar. Lets look at how this model performs and what issues may arise.
<br><br>

In [ ]:
preds_rd = np.load('tutorial_2/real_distribution_model/version_0/preds_real_distribution.npy')
grelu.visualize.plot_pred_scatter(
    preds=preds_rd,
    labels=test_ds_rd.get_labels(),
    density=True, # Color points by local density
    figsize=(3, 2.5), # width, height
    size=.5
)


In [ ]:
print_stats(preds_rd, test_ds_rd)

<br>
This real distribution model does a lot better when tested on an unseen real distribution testset (mse 2.011), when compared to the all peaks model (mse 13.51). However, one would expect the mse to be lower. The model has not learned enough about the extreme regions e.g. peak/non-peak regions, and has made more 'central' predictions closer to the mean. This does indicate potential underfitting due to the limited dataset size and complexity of the model. However, it does also highlight the problem with unbalanced data. <br><br>

While class imbalance is more closely associated with categorical models, the crux of the issue is, if we aim to identify and predict coverage levels, even more complicated models may still struggle with the extreme values. This is because when training a model on unbalanced data, it tends to become biased towards the majority class (in this case, non-peaks). In a regression setting, if 95% of the data points are "nonpeaks" and only 5% are "peaks," the model might end up predicting values close to the mean or median of the majority class more frequently to minimise the overall error, because it doesn’t encounter enough examples during training. <br><br>

The pitfall we are dealing with is one of <b>unbalanced classes</b>. The true percentage of peaks within the test data is around 5%. While the All Peaks model was trained on 100% peaks, this model was trained with the real distribution of peaks in chromosomes 1, 4, 5 (~5%). The overall mse has been reduced by narrowing the distribution of predictions towards the mean, resulting in inaccurate predictions for peaks. Rebalancing approaches such as undersampling the majority class, oversampling the minority class, and weighting are used to boost the performance on the minority class. In regression models aiming to predict peak coverage, such as this, it is common to undersample the amount of non-peaks. Lets look at the next model, the Balanced Model, which is trained on a 50-50 split of peaks and non-peaks utilising an undersampling approach.
<br><br>



Balanced Model
-

The Balanced Model is trained on an equal amount of peaks and non-peaks (15,000 samples).

In [ ]:
with open('datasets/train_ds_balanced.pkl', 'rb') as f:
    train_ds_bal = pickle.load(f)
    
with open('datasets/val_ds_balanced.pkl', 'rb') as f:
    val_ds_bal = pickle.load(f)
    
with open('datasets/test_ds_real_distribution.pkl', 'rb') as f:
    test_ds_bal = pickle.load(f)
    
train_labels = np.array(train_ds_bal.get_labels())
val_labels = np.array(val_ds_bal.get_labels())
test_labels = np.array(test_ds_bal.get_labels())

# Plot distributions for each dataset
plot_distribution(train_labels, 'Training Dataset')
plot_distribution(val_labels, 'Validation Dataset')
plot_distribution(test_labels, 'Test Dataset')

In [ ]:
preds_bal_rd = np.load('tutorial_2/balanced_model/version_0/preds_real_dist.npy')
grelu.visualize.plot_pred_scatter(
    preds=preds_bal_rd,
    labels=test_ds_rd.get_labels(),
    density=True, # Color points by local density
    figsize=(3, 2.5), # width, height
    size=.5
)

In [ ]:
print_stats(preds_bal_rd, test_ds_real_distribution)

<br>
The balanced model's prediction distribution is wider, indicating promise that a model balanced in peaks/non-peaks allows a better chance at predicting peaks accurately because they are more present in the training set. However the prediction results still indicate that the model is underfitting and not learning features of sequences that result in higher coverage values, and instead the model is biased towards predicting the mean of the training distribution. We can test this by plotting a prediction distribution curve.
<br><br>

In [ ]:
grelu.visualize.plot_pred_distribution(
    preds=preds_bal_rd,
    labels=test_ds_real_distribution.get_labels(),
    figsize=(3, 2.5), # width, height
    size=.5
)

As we can see, the prediction distribution is still too narrow failing to capture the underlying trends of the input sequences that result in different coverage values, indicating underfitting. Before we increase the amount of training data and model complexity, eventhough these models underfit, lets explore using a validation set of real peak/non-peak distributions from chromosome 3, and what happens if we don't split our datasets by chromosomes. As all of these models were trained on 15,000 datapoints using the same training and model parameters, we should be able to evaluate how these changes impact predictions, eventhough the models are underfitting. Firstly, lets use a validation set consisting of a real peak/non-peak distribution in chromosome 3.
<br><br>

Real Validation Model
-

In [ ]:
with open('datasets/train_ds_balanced.pkl', 'rb') as f:
    train_ds_rv = pickle.load(f)
    
with open('datasets/val_ds_real_distribution.pkl', 'rb') as f:
    val_ds_rv = pickle.load(f)
    
with open('datasets/test_ds_real_distribution.pkl', 'rb') as f:
    test_ds_rv = pickle.load(f)
    
train_labels = np.array(train_ds_rv.get_labels())
val_labels = np.array(val_ds_rv.get_labels())
test_labels = np.array(test_ds_rv.get_labels())

# Plot distributions for each dataset
plot_distribution(train_labels, 'Training Dataset')
plot_distribution(val_labels, 'Validation Dataset')
plot_distribution(test_labels, 'Test Dataset')

In [ ]:
preds_rv = np.load('tutorial_2/real_val_model/version_0/preds_real_dist.npy')
grelu.visualize.plot_pred_scatter(
    preds=preds_rv,
    labels=test_ds_rd.get_labels(),
    density=True, # Color points by local density
    figsize=(3, 2.5), # width, height
    size=.5
)

In [ ]:
print_stats(preds_rv, test_ds_real_distribution)

<br>
The real validation model has lower mse than balanced model. why? Well, while the model is trained on balanced data, validating on a real distribution impacts which model is chosen as the 'best model' based on the validation mse. These metrics are logged by gRelu within each model's subfolder. I have uploaded the metrics to Wandb as artifacts to easily visualise each model's run.

<img src="val_mse_comparison.png" alt="comparison of validation mse" width="600"/>

The real validation model's best checkpoint was chosen from epoch 3, where the validation mse is lowest. Since the real distribution validation set closely mirrors the true data distribution, it provides a more accurate reflection of the model's performance in the real distribution test set. Consequently, the real validation model is better at minimizing errors across the actual data distribution, leading to a lower overall MSE when evaluated on chromosome 2. While this model's 'better' performance likely does not hold statistically due to model complexity, it highlights issues regarding the potential trade-off between optimizing for a balanced dataset and achieving generalization in real-world scenarios, especially with **unbalanced data**.<br>

The choice between using a balanced validation set versus a validation set that reflects the real distribution of your data depends on what you prioritize: model performance on the true distribution or understanding model behavior on the minority ranges (peaks).

The pitfall relating to **distributional differences** occurs when the distributions of training and test sets differ from the prediction set. In the case of the Balanced Model, training and testing on datasets with balanced distributions, results in a lower MSE of 3.749 (same setting) than trying to predict on the real distribution of chromosome 2, 4.397 (different setting). While the aim is to train a model that generalises well, if the distributions of the training set and test set are similar, performance on a prediction set with a different distribution will be lower. It is therefore important that if we aim to predict on a real distribution of peaks/non-peaks, that we test and optimise our model on real distribution test set<br>

In [ ]:
test_metrics_bal = pd.read_csv('tutorial_2/balanced_model/version_0/test_metrics_bal.csv')
print(test_metrics_bal)

In [ ]:
test_metrics_bal_rd = pd.read_csv('tutorial_2/balanced_model/version_0/test_metrics_real_dist.csv')
print(test_metrics_bal_rd)

<br>

In the wider genomic context, these distributional differences can come from many sources such as inherent biological structures like chromosomal differences or applying models in different biological contexts. A <a href="https://www.nature.com/articles/s41576-021-00434-9" target="_blank">study</a> showed that the type of experiment that data comes from can change the distribution of the data. Models trained on one distribution have higher performance compared to others. Additionally, performance also drops when some models are applied to make predictions on a different species from the one on which they were trained. These differences in distributions is important to take into account when training models. <br>

Furthermore "distributional differences can arise from various sources, such as batch effects. If the training and test sets are a mixture of examples from every batch, performance on the testing set will be much higher than on a new batch. To fit a model that will generalize to new batches, training and test sets should be composed of different batches". <br>
<img src="distributional_differences.png" alt="distributional differences" width="700"/>

To simulate this distributional difference from batch effects, a model will be trained without splitting by chromosomes. Two models were trained on the real distribution of peaks/non-peaks, an Unsplit Model and a Comparison Model. The Unsplit Model was trained on chromosomes 1, 3, 4, 5, and 19, and validated/tested on different samples of the same chromosomes. The Comparison Model was trained on chromosomes 1, 4, 5, validated on chromosome 3, and tested on chromosome 19. Other training parameters and dataset sizes were kept constant. Chromosome 19 was chosen as the test chromosome because it has about a 20% peak rate compared to the other selected chromosomes peak rate of around 5%. In the context of batch effects, testing on the same batch (or in this case, the same chromosomes) does not challenge the model to generalize to new, unseen conditions (like data from a different batch or a chromosome with a different peak rate). As a result, the model might learn to capture batch-specific artifacts or characteristics, leading to high performance on the test set but poor generalization to new data.
<br><br>

In [ ]:
# Unsplit Model
test_metrics_unsplit = pd.read_csv('tutorial_2/unsplit_model/version_0/test_metrics_1.csv')
print(test_metrics_unsplit)

In [ ]:
# Comparison Model
test_metrics_compare = pd.read_csv('tutorial_2/compare_model/version_0/test_metrics_1.csv')
print(test_metrics_compare)

In [ ]:
### Models were run 5 times and a paired T-test was used to statistically test differences in MSE
### Subsequent run metrics are in the model subfolders

import glob
from scipy import stats

unsplit_files = glob.glob('tutorial_2/unsplit_model/version_0/test_metrics_*.csv')
compare_files = glob.glob('tutorial_2/compare_model/version_0/test_metrics_*.csv')

# Function to extract MSEs from a list of files
def load_mse_from_files(files):
    mse_list = []
    for file in files:
        df = pd.read_csv(file)
        mse_list.append(df['test_mse'].iloc[0])
    return np.array(mse_list)

# Load the MSEs for both models
mse_unsplit = load_mse_from_files(unsplit_files)
mse_compare = load_mse_from_files(compare_files)

# Performing the paired t-test
t_statistic, p_value = stats.ttest_rel(mse_unsplit, mse_compare)

print(f"T-statistic: {t_statistic}")
print(f"P-value: {p_value}")


<br>
The models were run 5 times and their MSE's were tested using a paired T-test to establish significance. Eventhough the validation and test sets used in the Unsplit Model were separated from the training set, the Unsplit Model has a better performance. In the context of batch effects, the Unsplit Model's performance is inflated because it has been trained and tested on data with similar distributional properties, effectively simulating a scenario where both the training and test sets come from the same batch and batch effects are present. This lack of true independence between the sets means the model is not challenged to generalize to new, unseen data. Creating models in this way does not test how well they generalise to unseen data and any predictions made on additional unseen data has to be scrutinized. Therefore, to avoid this pitfall training and test sets should be composed of different chromosomes and/or batches.

<br><br>

Using all chromosomes
-

The All Chromosomes Comparison Model was validated on chromosome 3, tested on chromosome 2 and trained on all the remaining autosomes balanced in peaks/non-peaks. This model changed only two parameters and uses roughly 10x the amount of data than the other models. The number of channels was decreased from 128 to 64, and the batch size was reduced from 256 to 128. This example is shown to highlight a 'better' model encorporating the design choices discussed previously in Tutorial 1.5 as well as this tutorial.

In [ ]:
preds_acc_bal = np.load('tutorial_2/all_chroms_compare_model/version_0/preds_balanced.npy')
grelu.visualize.plot_pred_scatter(
    preds=preds_acc_bal,
    labels=test_ds_bal.get_labels(),
    density=True, # Color points by local density
    figsize=(3, 2.5), # width, height
    size=.5
)

In [ ]:
grelu.visualize.plot_pred_distribution(
    preds=preds_acc_bal,
    labels=test_ds_bal.get_labels(),
    figsize=(3, 2.5), # width, height
    size=.5
)

In [ ]:
test_metrics_acc = pd.read_csv('tutorial_2/all_chroms_compare_model/version_0/test_metrics_bal.csv')
print(test_metrics_acc)

In [ ]:
with open('datasets/unsplit_allchroms/test_ds_compare.pkl', 'rb') as f:
    test_ds_acc = pickle.load(f)
    
preds_acc_rd = np.load('tutorial_2/all_chroms_compare_model/version_0/preds_real_dist.npy')
grelu.visualize.plot_pred_scatter(
    preds=preds_acc_rd,
    labels=test_ds_acc.get_labels(),
    density=True, # Color points by local density
    figsize=(3, 2.5), # width, height
    size=.5
)

In [ ]:
grelu.visualize.plot_pred_distribution(
    preds=preds_acc_rd,
    labels=test_ds_acc.get_labels(),
    figsize=(3, 2.5), # width, height
    size=.5
)

In [ ]:
test_metrics_acc_rd = pd.read_csv('tutorial_2/all_chroms_compare_model/version_0/test_metrics_real_dist.csv')
print(test_metrics_acc_rd)

<br>
The prediction distribution has improved immensely, suggesting that the model is better at capturing the underlying features of the input sequences that result in different coverage values, however signs of underfitting are still present, which makes sense as we lowered the number of channels and batch size. More importantly, another issue we face is that by balancing the training set with peaks and non-peaks, we have inadvertantly made the distribution of predictions more closely match the balanced training set instead of the unbalanced set. That is to say the mean of the predicted distribution is closer to the mean of the training set. This is common when re-balancing datasets, and  can be handled in recalibration based post-processing steps.

<br><img src="allchroms.png" alt="All chromosome model" width="600"/> <br>

Additionally, the validation MSE appears to be on a downward trend, however the validation pearson correlation plateaus further suggesting increasing the model complexity is crucial in the model understanding the relationship between sequence features and coverage values.
<br><br>

Model Improvements
-

Improving the model design, besides just increasing the model complexity, could include implementing cross-validation. In this case, balancing of the training set should take place within the training fold to ensure that the balanced data is evaluated on real distribution data in the cross-validation fold to avoid both the distributional differences and unbalanced classes pitfalls. An additional pitfall that has yet to be discussed is one of **biological dependencies**. These are extremely difficult to detect and can also inflate performance during the cross-validation stage. From a genomics context, when attempting to predict protein interactions, if not accounted for, models may have a higher performance in the cross-validation stage than on unseen proteins due to these dependencies. The general consensus is that group k-fold cross-validation is preferred to protect as much as possible against non-independent examples being mixed between training and test sets. Additionally, **leaky pre-processing** can be an issue especially when normalising or standardising genomic data. In general "any data transformation that looks at multiple examples together can be problematic" when performed outside of cross validation. The Unsplit Model, while also an example of distributional differences, leaks the overall distribution of the data between training and test sets creating biases and artificially inflating performance. While we applied the arcsinh transformation, on our dataset before splitting our data, the transformation does not inherently depend on the overall distribution of the data or any global statistics. <br><br>


Confounding effects and differences in sequencing depths
-
This portion of the tutorial on understanding confounding effects in genomic contexts is simplified from this <a href="https://www.nature.com/articles/s41576-021-00434-9" target="_blank">study</a> which used ENCODE data on the histone modification H3K27ac, processed in multiple read depths by Whalen et al. It perfectly highlights models using multiple experiment datasets of different sequence depths can face confounding effects. As explained in section 1.2.1 Representing epigenetic data, sequencing methods such as ATAC-Seq or ChIP-Seq experiments produce sequencing read data. When alligned to a reference genome, the reads are aggregated into regions where they ‘pile up’ to form peaks, indicating areas of significant biological activity or modification. Following peak calling, we obtain p-value or fold change coverage data. However, the problem lies within the sequencing process and how many times on average a region is sequenced (the sequencing depth). <br>

<br><img src="seq_depth.png" alt="Whalen et al. Sequencing Depth" width="1000"/> <br>

As seen in the image above, their results show that the signal p-values are higher for experiments with higher read depths, as regions were sequenced more on average. The important aspect of the effect is that the differences in signal p-values is non-linear. Furthermore, a higher sequencing depth means "that there are more peaks with higher values". The consequence of this being that if a machine learning model is trained to predict a signal in an experiment but then tested on another experiment, prediction discrepancies would systematically occur if the experiments have differing sequence depths. To mitigate this, it's crucial to either design a training approach that explicitly considers read depth or to reprocess the data and normalize for these differences. This can be done in many ways such as Total Count Normalization (TC), Counts Per Million (CPM) or even quantile normalisation.
<br><br>
I thought it would be interesting to repurpose Whalen et al's code to look at the data used in this tutorial, from ENCODE experiment <a href="https://www.encodeproject.org/experiments/ENCSR817LUF/" target="_blank">ENCSR817LUF</a> and compare it to another experiment targeting the same histone modification in a different cell. From ENCSR817LUF (brain tissue), the p-value bigwig we're using originates from raw read data with a sequencing depth of 36. Another experiment targets the same histone in a different cell (lung tissue) and has a sequencing depth of 76.
<br><br>

In [ ]:
!wget -nc https://www.encodeproject.org/files/ENCFF042SAY/@@download/ENCFF042SAY.bigWig #76 sequencing depth
!wget -nc https://www.encodeproject.org/files/ENCFF601VTB/@@download/ENCFF601VTB.bigWig #36 sequencing depth
!mv ENCFF042SAY.bigWig H3K36me3.76M.bigWig
!mv ENCFF601VTB.bigWig H3K36me3.36M.bigWig

In [ ]:
bw_filename = "H3K36me3.{}M.bigWig"
tracks = []

read_depths = 36, 76,
for reads in read_depths:
    filename = bw_filename.format(reads)
    bw = pyBigWig.open(filename)
    track = bw.values('chr2', 0, 1000000)
    track = numpy.nan_to_num(track)
    tracks.append(track)

In [ ]:
from matplotlib.gridspec import GridSpec

gs = GridSpec(nrows=1, ncols=5)

mid = tracks[-1].argmax()
start, end = mid - 6000, mid + 4000

plt.figure(figsize=(14, 3))

# Plot H3K27ac Signal Value At Differing Depths
plt.subplot(gs[:3])
plt.title("H3K36me3 Signal Value At Differing Depths Across Experiments", fontsize=14)
for i in range(len(read_depths)):  # Use len(read_depths) instead of 6
    plt.plot(range(start, end), tracks[i][start:end], label="{}M Reads".format(read_depths[i]))
plt.plot(range(start, end), np.zeros(end-start), color='0.7')
plt.legend(fontsize=10)
plt.ylabel("-log10 signal p-value")
plt.xlabel("Genomic Coordinate (chr2)")
plt.ylim(0, 50)
plt.xlim(start, end)

plt.tight_layout()
plt.show()

Looking at chromosome 2, our test set, it's interesting to see the different peak regions from each experiment (different cell types). As we are looking across experiments and sequencing depths, a lower sequenceing depth of 36 (blue) is not necessarily bound by a higher sequencing depth (orange). It does however, point out the need for normalising by sequencing depth in order to make comparisons on the absolute signal p-value peaks, further highlighting the importance of accounting for sequencing depth when creating models. 